In [1]:
from sklearn.ensemble import RandomForestClassifier

import utils


In [2]:
import os
data_folder_path = "./data"
print(os.listdir(data_folder_path))

x_train, y_train, x_test = utils.load_data(data_folder_path)

['test.csv', 'train.csv']


In [3]:
def cv_rf(X_train, y_train, X_valid, y_valid, h: dict):
    c = RandomForestClassifier(n_estimators = h.get('n_estimators', 100), criterion = h.get('criterion', "gini"), \
                               max_features = h.get('max_features', "sqrt"), \
                               random_state = 54321, n_jobs = -1) # set n_jobs to speed up by using more CPU cores
    m = c.fit(X_train, y_train)
    p = m.predict(X_valid)

    return utils.evaluate(y_valid, p)

In [4]:
hyperparameters = {"n_estimators": range(50, 1050, 50),
                   "criterion": ["gini", "entropy", "log_loss"],
                   "max_features": ["sqrt", "log2", None]}


In [5]:

pr = utils.kfold_cv(x_train, y_train, k = 10, H = list(utils.create_H(hyperparameters)), cv_fun = cv_rf, random_state = 12345)
pr

,accuracy,precision,recall,f,_h
0,0.928655,0.933654,0.914736,0.922570,"{'n_estimators': 50, 'criterion': 'gini', 'max..."
1,0.924336,0.930123,0.909169,0.917706,"{'n_estimators': 50, 'criterion': 'gini', 'max..."
2,0.952349,0.953277,0.945236,0.948900,"{'n_estimators': 50, 'criterion': 'gini', 'max..."
3,0.929833,0.934604,0.916295,0.923930,"{'n_estimators': 50, 'criterion': 'entropy', '..."
4,0.924860,0.931222,0.909385,0.918171,"{'n_estimators': 50, 'criterion': 'entropy', '..."
...,...,...,...,...,...
175,0.931798,0.937288,0.917946,0.925964,"{'n_estimators': 1000, 'criterion': 'entropy',..."
176,0.958372,0.959290,0.952107,0.955409,"{'n_estimators': 1000, 'criterion': 'entropy',..."
177,0.935987,0.940426,0.923615,0.930713,"{'n_estimators': 1000, 'criterion': 'log_loss'..."
178,0.931798,0.937288,0.917946,0.925964,"{'n_estimators': 1000, 'criterion': 'log_loss'..."


In [6]:
best_model = utils.get_best(pr)

0.9598121221686234
{'n_estimators': 400, 'criterion': 'entropy', 'max_features': None}


In [7]:
c = RandomForestClassifier(n_estimators = 400, criterion = 'entropy', \
                           max_features = None, \
                           random_state = 12345, n_jobs = -1) # set n_jobs to speed up by using more CPU cores
m = c.fit(x_train, y_train)
p = m.predict(x_test)

In [8]:
p_train = c.predict(x_train)
utils.evaluate(y_train, p_train)

{'accuracy': 1.0, 'precision': 1.0, 'recall': 1.0, 'f': 1.0}

In [19]:
print(p.size)
p

1910


array([0, 0, 0, ..., 0, 0, 1], dtype=int64)

In [16]:
import numpy as np
np.savetxt('predictions.txt', p, fmt='%d', newline='')